In [1]:
import numpy as np
from scipy.constants import G, c, pi
from copy import copy, deepcopy
import pylab, time, os, glob
import subprocess
from tempfile import mkstemp
from shutil import move, copyfile, rmtree

In [2]:
#Define Constants:

Msun = 1.98892e30    #kg

In [3]:
#Conversion factors:

kg_in_Msun = 1./Msun
m_in_Msun = c**2/G/Msun
s_in_Msun = c**3/G/Msun
print m_in_Msun

0.000677068614947


## TOV Solver - Isotropic Coordinates

In [4]:
#Polytropic Equation of State
def func_pressure(rho_c, Gamma0, K0, Gamma1=0, K1=0, rho_bndry=0, eostype="Polytrope"):
    if eostype=="polytrope":
        return K0*rho_c**Gamma0
    
    elif eostype=="PiecewisePolytrope":
        if rho_c < rho_bndry: return K0*rho_c**Gamma0
        else: return K1*rho_c**Gamma1
    
    elif eostype=="GammaLaw":
        return (Gamma0-1.)*rho_c
    
def func_central_density(press, Gamma0, K0, Gamma1=0, K1=0, press_bndry=0, eostype="Polytrope"):
    if eostype=="polytrope":
        if press<0:
            press = 0.0
        return (press/K0)**(1./Gamma0)

    elif eostype=="PiecewisePolytrope":
        if press < press_bndry: rhoc = (press/K0)**(1.0/Gamma0)
        else: rhoc = (press/K1)**(1.0/Gamma1)
        return rhoc
    
    elif eostype=="GammaLaw":
        return (press/K0)**(1./Gamma0)
    
def func_density(press, Gamma0, K0, Gamma1=0, K1=0, press_bndry=0, eostype="Polytrope"):
    
    if press<0:
        press=0.0
    
    if eostype=="polytrope":
        rho_c = func_central_density(press, Gamma0, K0, Gamma1, K1, press_bndry, eostype)
        return rho_c + press/(Gamma0-1.)
    elif eostype=="PiecewisePolytrope":
        rho_c = func_central_density(press, Gamma0, K0, Gamma1, K1, press_bndry, eostype)
        return rho_c
    elif eostype=="GammaLaw":
        return press/(Gamma0 - 1.)
            

        
# TOV Star in isotropic coordinates        
class bowentov_data(object):
    
    def __init__(self, initdata):     #eqstate, rho0, rho_atm, K0, n0, K1=0,  n1=0, rho_bndry=0):

        self.eos_type = initdata['eqstate'] 
        
        self.mass = initdata['init_mass']
        self.restmass = initdata['init_restmass']
        self.riso = initdata['r_init']
        self.theta = initdata['theta']
        self.thetader = initdata['thetader']
        self.psi = initdata['psi']
        self.psider = initdata['psider']
        self.verbose = initdata['verbose']

        if self.eos_type=="polytrope":
            self.K0 = initdata['K0']
            self.Gamma0 = 1.+(1./initdata['n0'])
            self.K1 = 0.0
            self.Gamma1 = 0.0
        
        elif self.eos_type=="PiecewisePolytrope":
        
            self.K0 = initdata['K0']
            self.Gamma0 = 1.+(1./initdata['n0'])
            self.K1 = initdata['K1']
            self.Gamma1 = 1.+(1./initdata['n1'])
            
        elif self.eos_type=="GammaLaw":
            
            self.Gamma0 = (1. + 1./initdata['n0'])
            self.K0 = initdata['K0']
            self.Gamma1 = 0.0
            self.K1 = 0.0
           
        self.press_bndry = 0.0
        rho_bndry = 0.0
        rho_atm = initdata['rho_atm']
        self.press_atm = func_pressure(rho_atm, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype=self.eos_type)


        if initdata['tov_type']=='tov_bowen':
            rho_c = initdata['rho0']
            self.press = func_pressure(rho_c, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype = self.eos_type)
        elif initdata['tov_type'] =='invtov_bowen':
            self.press = func_pressure(rho_atm, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype=self.eos_type)
           
          
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        
        if self.verbose>0:
            print("Initial Data : \n Pressure = %g \n Density = %g \n Central Density = %g \n Psi = %g \n Theta = %g \n Isotropic Radius = %g"
                  %( self.press, rho, rho_c, self.psi, self.theta, self.riso))
        
        
    #Before calling derivatives update the pressure, density, r and m
    def func_restmassder(self, r):

        rhoc = func_central_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        restmass_der = 4.*np.pi*rhoc*(r**2.)*(self.psi**6.)
        return restmass_der
    
    def func_admmassder(self, r):

        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        admmass_der = 4.*np.pi*rho*(r**2.)*(self.psi**5.)
        return admmass_der
    
    def func_pder(self, r):
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
         
        if r==0.:
            pder = 0.
        else:
            pder = -1.*(rho + self.press)*((self.thetader/self.theta) - (self.psider/self.psi))
        
        return pder
    
    def func_psider(self, r):
        
        return self.psider                                        
                                           
    def func_psiderder(self, r):
        
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        psi_dd = -2.*pi*rho*self.psi**5
        if r==0:
            psi_dd = psi_dd + 4.*pi*rho/3.
        else:
            psi_dd = psi_dd + (-1.)*2.*self.psider/r
                                           
        return psi_dd 
    
    def func_thetader(self, r):
                                           
        return self.thetader
                             
    def func_thetaderder(self, r):
        
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        theta_dd = 2.*pi*(rho + 6.*self.press)*self.theta*(self.psi**4.)
        if r==0:
            theta_dd = theta_dd - 4.*pi*(rho + 6.*self.press)/3.
        else:
            theta_dd = theta_dd -2.*self.thetader/r
         
        return theta_dd                                  
                                           
    def __add__(self, star2):
        
        y = copy(self)
        y.press = self.press + star2.press
        y.mass = self.mass + star2.mass
        y.restmass = self.restmass + star2.restmass
        y.theta = self.theta + star2.theta                                  
        y.psi = self.psi + star2.psi
        y.thetader = self.thetader + star2.thetader                                  
        y.psider = self.psider + star2.psider                                   
        y.riso = self.riso + star2.riso
                                           
        return y
        
    def __mul__(self, k):
        
        y = copy(self)
        y.press = k*self.press
        y.mass = k*self.mass
        y.restmass = k*self.restmass
        y.psi = k*self.psi
        y.theta = k*self.theta
        y.psider = k*self.psider
        y.thetader = k*self.thetader                                   
        y.riso = k*self.riso

        return y
 
    def __rmul__(self, k):
        y = copy(self)
        y.press = k*self.press
        y.mass = k*self.mass
        y.restmass = k*self.restmass
        y.psi = k*self.psi
        y.theta = k*self.theta
        y.psider = k*self.psider
        y.thetader = k*self.thetader                                    
        y.riso = k*self.riso
      
        return y
    
    
    
class bowenstar:
    
    def __init__(self, initdata ):
        self.star = bowentov_data( initdata)
        self.delta_r = initdata['delta_r']
        self.tov_type = initdata['tov_type']
        self.verbose = initdata['verbose']
        
    def func_rk4_rhs(self, r, tov_star):
    
        y = deepcopy(tov_star)
        
        y.press = tov_star.func_pder(r)
        y.mass = tov_star.func_admmassder(r)
        y.restmass = tov_star.func_restmassder(r)
        y.psi = tov_star.func_psider(r)
        y.psider = tov_star.func_psiderder(r)    
        y.theta = tov_star.func_thetader(r)
        y.thetader = tov_star.func_thetaderder(r)                                       
        y.riso = 1.
        y.K = 0.
        y.Gamma = 0.
        
        return y
    
    def rungekuttamethod(self, tov_star, r):
        
        h = copy(self.delta_r)
        st = time.time()
        
        y = deepcopy(tov_star)
        if self.verbose==2:
            print('Copy the class object in %g'%(time.time() - st))
            st = time.time()
        
        k1 = self.func_rk4_rhs(r, y)
        if self.verbose==2:
            print('k1 done in %g'%(time.time() - st))
            st = time.time()
        
        k2 = self.func_rk4_rhs(r+h/2., y + (h/2.)*k1)
        if self.verbose==2:
            print('k2 done in %g'%(time.time() - st))
            st = time.time()
        
        k3 = self.func_rk4_rhs(r+h/2., y + (h/2.)*k2)
        if self.verbose==2:
            print('k3 done in %g'%(time.time() - st))
            st = time.time()
        
        k4 = self.func_rk4_rhs(r+h, y+h*k3)
        if self.verbose==2:
            print('k1 done in %g'%(time.time() - st))
            st = time.time()
        
        tov_star = tov_star + (h/6.)*(k1 + 2.*k2 + 2.*k3 + k4)
        rn = r + h
        if self.verbose==2:
            print('tovstar done in %g'%(time.time() - st))
            st = time.time()
        
        return [rn,tov_star]

    
    
    def tovsolver(self):
        
        i= 1
        press, admmass, restmass, psi, psider, theta, thetader,riso = [], [], [], [], [], [], [], []
        #rho_central = func_central_density(self.star.press, self.star.Gamma0, self.star.K0, self.star.Gamma1, self.star.K1, self.star.press_bndry, eostype = self.star.eos_type)
        #self.star.m = 4.0*np.pi*r**2*rho_central
        r = self.star.riso

        press.append( self.star.press)
        admmass.append(self.star.mass)
        restmass.append(self.star.restmass)
        psi.append(self.star.psi)
        psider.append(self.star.psider)
        theta.append(self.star.theta)
        thetader.append(self.star.thetader)                            
        riso.append(r)
            
        if self.verbose==1:
            print("Initial Data: Radius = %g, delta_r = %g, Pressure = %g, rho_c = %g \n"%(self.star.riso, self.delta_r, self.star.press, rho_central) )
        
        while (self.star.riso>=0):#np.abs(self.delta_r)):
      
            r, self.star = self.rungekuttamethod(self.star,r)
            press.append(self.star.press)
            admmass.append(self.star.mass)
            restmass.append(self.star.restmass)
            psi.append(self.star.psi)
            psider.append(self.star.psider)                            
            theta.append(self.star.theta)
            thetader.append(self.star.thetader)                            
            riso.append(self.star.riso)
            i=i+1
            
            if  i>500000:
                if self.verbose>0:
                    print("TOV solver not working correctly. Broken by force. ")
                break
                
            
            if (self.star.press<0):
                if self.verbose>0:
                    print("Pressure has become negative at i=%d."%i)
                break
            
        neg_press_idx = np.where(np.asarray(press)<0)
        
        if (np.size(neg_press_idx)>0):          
            
            if self.verbose>0:
                print ("Pressure goes negative, Pressure = %g at r=%g"%(press[press<0], riso[press<0]))
            last_idx=np.amin(neg_press_idx)
                
            press = np.asarray(press[:last_idx])
            admmass = np.asarray(admmass[:last_idx])
            restmass = np.asarray(restmass[:last_idx])
            psi = np.asarray(psi[:last_idx])
            psider = np.asarray(psider[:last_idx])
            theta = np.asarray(theta[:last_idx])
            thetader = np.asarray(thetader[:last_idx])                                   
            riso = np.asarray(riso[:last_idx])
            
        else:
            press = np.asarray(press)
            admmass = np.asarray(admmass)
            restmass = np.asarray(restmass)
            psi = np.asarray(psi)
            psider = np.asarray(psider)
            theta = np.asarray(theta)
            thetader = np.asarray(thetader)                                   
            riso = np.asarray(riso)

            
        
        psi_rescale = psi[-1] + psider[-1]*riso[-1]
        theta_rescale = theta[-1] + thetader[-1]*riso[-1]
        
        theta = theta/theta_rescale
        psi = psi/psi_rescale
        riso = riso*psi_rescale**2.
        lapse = theta/psi
        admmass = admmass*psi_rescale
        if self.verbose>0:
            print("\n \n Final Parameters")
            print("Radius of Star = %g"%riso[-1])
            print("Stellar  ADM mass = %g"%admmass[-1])
            print("Stellar  Rest Mass = %g"%restmass[-1])
            print("Conformal Factor at r=0 = %g and at Rstar = %g"%(psi[0],psi[-1]))
            print("Theta at r=0 = %g and at Rstar = %g"%(theta[0],theta[-1]))
            print("Pressure at r=0 = %g and at Rstar = %g"%(press[0], press[-1]))
            
        
        
        if (len(press)>1):
            return [press, admmass, restmass, psi, psider, theta, thetader, riso]
        
        else:
            print("TOV integrator failed")
            return [press, admmass, restmass, psi, psider, theta, thetader, riso]
        
       

## Set the initial parameters here

In [30]:
#Set the desired mass ratio, Total mass (in Msun) and initial separation (in total mass)
q = 3. 
Mstar =  1.35                  # Msun
M_Msun = Mstar*(1+q)          # Msun
initsep_Mtotal = 9.         # Total Mass

# EOS parameters
eos_type = "polytrope"
gamma_poly = 2.
n = 1./(gamma_poly-1)
mbar_star =  0.25/4.34027777777778**0.5#0.25/2.50360519147573**0.5        #Mass in polytropic units

#The relationship between mbar and rhobar is not one-to-one hence, depending on the 
# desired compactness of star, choose a rhobar value

rhobar=0.05  #should be less than actual rhobar value

K_poly =  (Mstar/mbar_star)**(2./n) #123.641             # Msun**(2.)  
K_poly_M = K_poly/M_Msun**2
print("Polytropic Constant = %g Msun = %.10g M"%(K_poly, K_poly_M))

# Spin Parameters  
ax_bh = ay_bh = az_bh = 0.
ax_ns = ay_ns = az_ns = 0.

#Refinement levels - 
nlvl_in_star = 0
nlvl_outside_src = 7
npts_cover_star = 50
npts_cover_bh = 30 

#Scaling Factor - 
#change this to total mass if you want M_adm=1 else let it be 1 if you want everything in solar mass units
Mscale_in_Msun = M_Msun  #Msun


##############################################
##     Do Not change from here             ###
##############################################

# Compute masses of black hole and neutron star in terms of total mass and solar mass
m_bh = q/(1.+q)
m_ns = 1./(1+q)

m_bh_Msun = m_bh*M_Msun
m_ns_Msun = m_ns*M_Msun
print("Initial Desired Masses: BH mass = %g Msun, NS mass = %.10g Msun" %(m_bh_Msun, m_ns_Msun))
print("Initial Desired Masses (K=1): BH mass = %g Msun, NS mass = %.10g Msun" %(m_bh_Msun/K_poly**0.5, m_ns_Msun/K_poly**0.5))

#Compute separation in km
initsep_M = initsep_Mtotal*M_Msun/Mscale_in_Msun
initsep_km = initsep_M*Mscale_in_Msun/m_in_Msun/1000.

print("Initial Desired Separation = %g km = %g Msun"%(initsep_km, initsep_M*Mscale_in_Msun))



#Spin in spherical units - 

def spherical_coord(x,y,z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y,x)
    if r==0: theta = 0.
    if x==0: phi = 0.
    return r, theta, phi

a_bh_mag, a_bh_theta, a_bh_phi = spherical_coord(ax_bh, ay_bh, az_bh)
a_ns_mag, a_ns_theta, a_ns_phi = spherical_coord(ax_ns, ay_ns, az_ns)

Polytropic Constant = 126.563 Msun = 4.340277778 M
Initial Desired Masses: BH mass = 4.05 Msun, NS mass = 1.35 Msun
Initial Desired Masses (K=1): BH mass = 0.36 Msun, NS mass = 0.12 Msun
Initial Desired Separation = 71.78 km = 48.6 Msun


/localdata/bkhamesra3/softwares/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars


## Obtain Initial guesses for density and radius for above massive stars

In [31]:

def construct_tov(rho_init):
    """Set up a Isotropic TOV star for a given central density
    """
    
    initdata = {}
    initdata['eqstate'] = eos_type

    gamma = gamma_poly
    K =  K_poly                                        # Msun**(2.)  

    rhoc = rho_init                                      # kg/m**3
    rhoc = rhoc/((m_in_Msun**3)*Msun)                    # Msun
   
    rho_atm = rhoc*1e-10
    press_atm = K*rho_atm**gamma

    n = 1./(gamma-1)
    delta_r = 1e-3                                       
    verbose=0

    initdata['init_mass'] = 0.
    initdata['init_restmass'] = 0.
    initdata['r_init'] = 0.
    initdata['theta'] = 1.0
    initdata['psi'] = 1.
    initdata['psider'] = 0.
    initdata['thetader'] = 0.

    initdata['K0'] = K
    initdata['n0'] = n
    initdata['K1'] = 0.
    initdata['n1'] = 0.

    initdata['rho_atm'] = rho_atm
    initdata['rho0'] = rhoc
    initdata['tov_type']='tov_bowen'
    initdata['delta_r'] = delta_r
    initdata['verbose'] = verbose

    if verbose>0:
        print("Polytropic Constants: K = %.3f (Msun^2), Gamma = %.3f"%(K, gamma))
        print("Stellar Density: rhoc = %g(/Msun^2), Pressure = %g(/Msun^2) \n"%(rhoc, K*rhoc**gamma))

    tov_bowen = bowenstar(initdata)

    press_iso, admmass_iso, restmass_iso, psi_iso, psider_iso, theta_iso, thetader_iso, r_iso = tov_bowen.tovsolver()

    if press_iso[-1]<0:
        #print('Pressure at last index is negative \n')
    
        press_iso = press_iso[:-1]
        admmass_iso = admmass_iso[:-1]
        restmass_iso = restmass_iso[:-1]
        psi_iso = psi_iso[:-1]
        psider_iso = psider_iso[:-1]
        theta_iso = theta_iso[:-1]
        thetader_iso = thetader_iso[:-1]
        r_iso = r_iso[:-1]

    radius_iso = r_iso[-1]
    mass_iso = 2*radius_iso*(psi_iso-1)
    print("Rest Mass = %g, ADM mass1 = %g"%(restmass_iso[-1], mass_iso[-1]))
    return radius_iso, mass_iso[-1], restmass_iso[-1]




In [32]:
#Isotropic Radius of BH

radius_bh_iso = m_bh_Msun/2.
#radius_bh_sch = radius_bh_iso*(1. + m_bh_Msun/2./radius_bh_iso)**2
print("BH radius (isotropic) = %g Msun"%radius_bh_iso)

BH radius (isotropic) = 2.025 Msun


In [33]:
#Find the approximate initial density and radius guess for Broyden 

# Initial Guess Density - This set up based on the relations between rhobar, Rbar and Mbar.

# Choosing rhobar - The basic idea is to find the stellar configuration with small radius 
# so that high resolution of stellar grid can be achieved with small number of points i.e. 
# npts_star = radius/dx_star, Since dx_star is generally fixed ~250 metres, number of 
# points covering star would depend on size of the star which we wish to keep minimum 
# to decrease the memory usage. 


rho_Msun = rhobar/(K_poly**n)
rhoc = rho_Msun*((m_in_Msun**3)*Msun)
print rhoc

def get_densityguess(rho_init_old, dmbyr_old, dm_prev, rad_old, i=0):
    
    #Find radius and mass 
    var_rad_tov, var_admmass_tov, var_restmass_tov = construct_tov(rho_init_old)
    
    #Readjust the density - need to modify to bisection method
    dm_target = m_ns_Msun - var_admmass_tov
    dm_sq = (dm_target/var_rad_tov - dmbyr_old)
    
    fac=0.5
    
    rho_init_new = rho_init_old + (dm_target/fac)*rho_init_old
    if rho_init_new <0:
        raise ValueError("Density cannot be negative")
    i+=1
    
    if np.abs(dm_target)< 1e-4 :
        print("\n Density Obtained = %g and Radius = %g Msun with mass difference = %g"%(rho_init_new, var_rad_tov, dm_target))  
        return [rho_init_new,  var_rad_tov, var_restmass_tov, var_admmass_tov]
    elif np.abs(dm_target)>np.abs(dm_prev) and i>2:
        print("\n Delta M value started increasing, so stopping the iterations. rhoc = %g, delta M = %g"%(rho_init_new, dm_target))
    #    return [rho_init_old,rad_old ] 
    #elif ((i>1) and (dm_sq>0)):
    #   print(" Mass difference started increasing. Density  = %g, Radius - %g, delta_M = %g, dM/R = %g, delta(dM/R) = %g"%(rho_init, rad, dm_target, dm_target/rad, dm_sq))   
    else:
        print("Density  = %g, Radius = %g Msun, delta_M = %g, delta M/R = %g, delta(dM/R) = %g"%(rho_init_new, var_rad_tov, dm_target, dm_target/var_rad_tov, dm_sq))
        return get_densityguess(rho_init_new, dm_target/var_rad_tov, dm_target, var_rad_tov, i)
        
        
        
rho_init, radius_iso,  restmass_ns,admmass_ns  = get_densityguess(rhoc, 0, 100., 0, i=0)   


rho_ns_init = rho_init
radius_ns_iso = radius_iso
radius_ns_sch = radius_ns_iso*(1. + m_ns_Msun/2./radius_ns_iso)**2

2.43882322899e+17
Rest Mass = 1.00829, ADM mass1 = 0.969294
Density  = 4.29577e+17, Radius = 11.4785 Msun, delta_M = 0.380706, delta M/R = 0.0331669, delta(dM/R) = 0.0331669
Rest Mass = 1.42799, ADM mass1 = 1.34623
Density  = 4.32816e+17, Radius = 10.1589 Msun, delta_M = 0.0037697, delta M/R = 0.000371075, delta(dM/R) = -0.0327959
Rest Mass = 1.43362, ADM mass1 = 1.35116
Density  = 4.31809e+17, Radius = 10.1392 Msun, delta_M = -0.00116357, delta M/R = -0.000114759, delta(dM/R) = -0.000485834
Rest Mass = 1.43187, ADM mass1 = 1.34963
Density  = 4.32125e+17, Radius = 10.1457 Msun, delta_M = 0.00036572, delta M/R = 3.60469e-05, delta(dM/R) = 0.000150806
Rest Mass = 1.43242, ADM mass1 = 1.35011
Density  = 4.32026e+17, Radius = 10.1434 Msun, delta_M = -0.000114293, delta M/R = -1.12678e-05, delta(dM/R) = -4.73147e-05
Rest Mass = 1.43225, ADM mass1 = 1.34996

 Density Obtained = 4.32057e+17 and Radius = 10.1446 Msun with mass difference = 3.57624e-05


In [34]:
print(restmass_ns)

1.4322503691094097


In [35]:
rhof_Msun =  rho_init/((m_in_Msun**3)*Msun)
rhof_bar = rhof_Msun*K_poly**n
print("Finalized Initial Density = %g, rhobar = %g"%(rho_init, rhof_bar))


Finalized Initial Density = 4.32057e+17, rhobar = 0.088579


In [36]:
# Rescale the radius in units of total mass and in metres and print the source information
# As we use isotropic coordinates, it is better to only look at all quantities in these 
# coordinates. Also, conversion of tidal radius from isotropic to Schwarzschild is not exactly clear

radius_bh_iso_M = radius_bh_iso/Mscale_in_Msun
radius_ns_iso_M = radius_ns_iso/Mscale_in_Msun

#radius_bh_sch_m = radius_bh_sch/m_in_Msun
#radius_ns_sch_m = radius_ns_sch/m_in_Msun

radius_bh_iso_m = radius_bh_iso/m_in_Msun
radius_ns_iso_m = radius_ns_iso/m_in_Msun


rtidal_iso_Msun = 2.4*(q**(-2./3))*(radius_ns_iso/m_ns_Msun)*m_bh_Msun
rtidal_iso_m = rtidal_iso_Msun/m_in_Msun

#rtidal_sch_Msun = 2.4*(q**(-2./3))*(radius_ns_sch/m_ns_Msun)*m_bh_Msun
#rtidal_m_sch = rtidal_sch_Msun/m_in_Msun

r_isco_sch = 6.*m_bh_Msun
r_isco_sch_m = r_isco_sch/m_in_Msun
r_isco_iso_m = 0.5*(r_isco_sch - m_bh_Msun + r_isco_sch*(1. - 2.*m_bh_Msun/r_isco_sch)**0.5)/m_in_Msun

#print("Tidal Radius (Schw Coord) = %g km, ISCO = %g km"%(rtidal_m_sch/1000., r_isco_m/1000.))
print("Tidal Radius (Iso Coord) = %g km, ISCO = %g km"%(rtidal_iso_m/1000., r_isco_iso_m/1000.))

print("BH:")
print("\t Radius (Isotropic) = %.10g km \n \t Mass = %g Msun\n"\
      %(radius_bh_iso_m/1000,  m_bh_Msun))

print("Star:")
print("\t rhoc = %.15g kg/m^3 \n \t Radius (Isotropic) = %.10g km \n \t Mass = %g Msun"\
      %(rho_ns_init, radius_ns_iso_m/1000, m_ns_Msun))
print("\t Compactness  = %g \n"%((radius_ns_iso/m_ns_Msun)**-1))

print("Separation = %g M (%g km)"%(initsep_M, initsep_km))
if initsep_M < (radius_bh_iso_M + radius_ns_iso_M)*1.2:
    raise ValueError("Initial separation too small compared with sizes of star. Please increase the separation!")


Tidal Radius (Iso Coord) = 51.8626 km, ISCO = 29.6062 km
BH:
	 Radius (Isotropic) = 2.990834245 km 
 	 Mass = 4.05 Msun

Star:
	 rhoc = 4.32056941491191e+17 kg/m^3 
 	 Radius (Isotropic) = 14.98314427 km 
 	 Mass = 1.35 Msun
	 Compactness  = 0.133076 

Separation = 9 M (71.78 km)


In [37]:
#Quantities in polytropic units (for paper/comparisons)
n = 1./(gamma_poly-1)
Rbar_ns_check = radius_ns_iso/(K_poly**(n/2.))
Rbar_bh_check = radius_bh_iso/(K_poly**(n/2.))

Mbar_ns_check = admmass_ns/(K_poly**(n/2.))
Mbar_restns_check = restmass_ns/(K_poly**(n/2.))
Mbar_bh_check = m_bh_Msun/(K_poly**(n/2.))

print("Configuration Details (in polytropic units) - ")
print("Mass Ratio - %g (%g)"%(Mbar_bh_check/Mbar_ns_check, m_bh_Msun/m_ns_Msun))
print("ADM mass of star = %g Rest Mass of Star = %g"%(Mbar_ns_check, Mbar_restns_check))
print("ADM mass of BH = %g "%(Mbar_bh_check))
print("Total ADM Mass = %g"%(M_Msun/K_poly**(n/2.)))

print("Compactness = %g"%(Mbar_ns_check/Rbar_ns_check))
print("Central density = %g"%(rhof_bar))
print("Rbar_star = %g, Rbar_bh = %g "%(Rbar_ns_check, Rbar_bh_check))

radius_ns_sch_m = radius_ns_sch/m_in_Msun
rbar_sch_ns_check = radius_ns_sch/(K_poly**(n/2.))
print("Radius (Schw) = %g m, Rbar (Schw) = %g"%(radius_ns_sch_m, rbar_sch_ns_check))
print("Compactness (Schw Radius) = %g"%(Mbar_ns_check/rbar_sch_ns_check))

Configuration Details (in polytropic units) - 
Mass Ratio - 3.00008 (3)
ADM mass of star = 0.119997 Rest Mass of Star = 0.127311
ADM mass of BH = 0.36 
Total ADM Mass = 0.48
Compactness = 0.133072
Central density = 0.088579
Rbar_star = 0.901744, Rbar_bh = 0.18 
Radius (Schw) = 17043.4 m, Rbar (Schw) = 1.02574
Compactness (Schw Radius) = 0.116986


In [ ]:
#print("C2 = {}, C2_sch = {}".format(0.1666666666666/0.671827999999, 0.1666666666666/0.8488313071))
#print("C4 = {}, C4_sch = {}".format(0.1666666666666/0.77269199999, 0.1666666666666/0.94834600501))
#print("C1 = {}, C1_sch = {}".format(0.16666666666/0.9432879999996, 0.16666666666/1.1173166223))

#print("C3 = {}, C3_sch = {}".format(0.1666666666666/1.226709999, 0.1666666666666/1.39903769))


In [ ]:
# 14675.54/65
 #11284/50  
9243.97/50

In [ ]:
# Common Functions: 

def read_data(mparfile, data):

    datafile = open(mparfile,'r')
    datafile.seek(0)
    for line in datafile.readlines():
        if data in line:
            break
    line = line.split()
    data_value = (line[-1]).split(';')[0]
    datafile.close()
    return data_value

def replace_data(file_path, pattern, subst, verbose=False):
    
    fh, abs_path = mkstemp()
    with os.fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            pattern_found=0
            for line in old_file:
                if pattern_found==1:
                    new_file.write(line)#.replace(pattern, subst))
                elif pattern in line:
                    if verbose: print pattern_found
                    pattern_found=1
                    if verbose: print pattern_found , line
                    line_split = line.split('#')[0]
                    line_parts = (line_split.split('=')[-1]).split()
                    value = line_parts[0]
                    new_file.write(line.replace(value, subst))
                else:
                    new_file.write(line)#.replace(pattern, subst))
                
    #Remove original file
    os.remove(file_path)
    #Move new file
    move(abs_path, file_path)


def replace_data_mparfile(file_path, pattern, subst, verbose=False):
    
    fh, abs_path = mkstemp()
    with os.fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            pattern_found=0
            for line in old_file:
                if pattern_found==1:
                    new_file.write(line)#.replace(pattern, subst))
                elif pattern in line:
                    if verbose: print pattern_found
                    pattern_found=1
                    if verbose: print pattern_found , line
                    line_split = line.split('#')[0]
                    line_parts = (line_split.split('=')[-1]).split()
                    value = line_parts[0]
                    
                    line_split = line.split()
                    idx = line_split.index(value)
                    line_split[idx] = subst
                    line_split.append('\n')
                    new_file.write(' '.join(line_split))
                else:
                    new_file.write(line)#.replace(pattern, subst))
                
    #Remove original file
    os.remove(file_path)
    #Move new file
    move(abs_path, file_path)
                

                
    

In [ ]:
#Create mpar file for PNevo

#Enter the directory of PNEvo - Change this path
os.chdir('/nethome/bkhamesra3/Cactus_branches/Cactus_2018/repos/tools/PNevo/')

params = ['par.initial_sep', 'par.final_sep', 'par.MM', 'par.mass_ratio', 'par.a1x', 'par.a1y', 'par.a1z', \
         'par.a2x', 'par.a2y', 'par.a2z', 'par.manual_p0', 'par.px0', 'par.py0', 'par.pz0', 'par.fixed_dt', \
         'par.dtfac', 'par.spinspin', 'par.radrxn', 'par.out_time', 'par.out_baumfile', 'par.out_rpar_head']

#Provide the initial parameters in terms of total mass =1
par_initvalues = {'par.initial_sep':100,           # Initial Separation
                  'par.final_sep': initsep_Mtotal,     # Final Separation
                  'par.MM':1,                     # Total Mass (remains 1)
                  'par.mass_ratio':q,             # Mass Ratio of binary
                  'par.a1x':ax_bh,                # Dimensionless spin of BH1
                  'par.a1y':ay_bh,
                  'par.a1z':az_bh,
                  'par.a2x':ax_ns,              # Dimensionless spin of BH2
                  'par.a2y':ay_ns,
                  'par.a2z':az_ns, 
                  'par.manual_p0':0,    # Change to 1 if manually specifying the momentum of BH1 (p_BH2 = -1*p_BH1)
                  'par.px0':0.0,          # Momentum of BH1 (If 0, will be evaluated by PNevo) 
                  'par.py0':0.0, 
                  'par.pz0':0,
                  'par.fixed_dt':0,     # If 1, timestep = fixed_dt. If 0, time step adjusted internally based on separation
                  'par.dtfac':25,       # Timestep of evolution. Set fixed_dt to 1 if you want to specify the timestep  here.
                  'par.spinspin':1,     # Account for spin-spin interactions (default is 1)
                  'par.radrxn':1,       # Account for Radiation reaction (default is 1)
                  'par.out_time':0,     # Output time
                  'par.out_baumfile':1, # Produced Maya parfile for use with Baum Solver (default =1)
                  'par.out_rpar_head':1} #Create file with all initial parameters for rpar file (default = 1)


if os.path.exists('./initdata_NSBH.mpar'):
    print("File found")
for p in params:
    replace_data_mparfile('initdata_NSBH.mpar', p, str(par_initvalues[p]))
    



In [ ]:
#Run Post Newtonian evolution script - PNevo, and save the momenta from output
output = subprocess.check_output(["./PNevo", "initdata_NSBH.mpar"])
print output
for line in output.splitlines():
   
    if "$ppx" in line:
        px_bh = float((line.split("=")[-1]).split(';')[0])
    
    elif "$ppy" in line:
        py_bh = float((line.split("=")[-1]).split(';')[0])


#Compute momentum in Msun units
px_bh_Msun = px_bh*M_Msun
py_bh_Msun = py_bh*M_Msun
px_star_Msun = -1.*px_bh*M_Msun
py_star_Msun = -1.*py_bh*M_Msun

print("x-Momentum of object 1: {}".format(px_bh_Msun))
print("y-Momentum of object 1: {}".format(py_bh_Msun))
print("x-Momentum of object 1: {}".format(px_star_Msun))
print("y-Momentum of object 1: {}".format(py_star_Msun))

## Generate rpar file for Initial Data (Broyden)

In [ ]:
#Path where mixed binary files are stored - 

#Change this
os.chdir('/nethome/bkhamesra3/Cactus_branches/Cactus_2019/par/NSBH/')

#Create a copy of rpar file for generating broyden parfile
copyfile('NSBH_Updated.rpar', 'NSBH_ID_Python.rpar')

In [ ]:
#Modify the NSBH rpar file to create parfile for Broyden:

nsbh_id_rpar = 'NSBH_ID_Python.rpar'

#Check the compact object type is BH and TOV
parobject1 = read_data(nsbh_id_rpar, 'compact_object1')
parobject2 = read_data(nsbh_id_rpar, 'compact_object2')

if not(parobject1=="\"BH\"") or not(parobject2=="\"TOV\"") :
    raise ValueError("This rpar file in not for NSBH. Please check the correct rparfile")


init_params = { "my $M_in_Msun": Mscale_in_Msun,
                "my $mq": q,
                "my $mStar_Msun": m_ns_Msun,
                "my $rStar_m": radius_ns_iso_m,
                "my $rhoStar_central_kgm3": rho_ns_init,
                "my $eos_type": "\"%s\""%eos_type,
                "my $eos_k_Msun": K_poly,
                "my $eos_gamma": gamma_poly,
                "my $params_from_broyden": 0,
                "my $momentum_from_PN" : 1,
                "my $init_separation_m": initsep_km*1000.,
                "my $P1x_PN": px_bh_Msun,
                "my $P1y_PN": py_bh_Msun,
                "my $P2x_PN": px_star_Msun,
                "my $P2y_PN": py_star_Msun,
                "my $a_BH_mag" : a_bh_mag, 
                "my $a_BH_theta" : a_bh_theta, 
                "my $a_BH_phi" : a_bh_phi,
                "my $a_Star_mag" : a_ns_mag, 
                "my $a_Star_theta" : a_ns_theta, 
                "my $a_Star_phi" : a_ns_phi,
                
                # Below parameters are for evolution only, nlevels and npoints are fixed for Broyden inside rpar
                "my $nlevels_within_star" : nlvl_in_star,
                "my $nlevels_beyond_src" : nlvl_outside_src,  
                "my $npoints_rad_star" : npts_cover_star,
                "my $npoints_rad_bh" : npts_cover_bh
              }

#Replace the parameters in rpar file
for key in sorted(init_params.keys()):
    print("{}  = {}".format(key[4:], init_params[key]))

    if key=="my $rho1_central_kgm3" or key=="my $eos_type":
        replace_data(nsbh_id_rpar, key, str(init_params[key])+';', verbose=True)
    else:
        #replace_data(nsbh_id_rpar, key, str(init_params[key])+';', verbose=False)
        replace_data(nsbh_id_rpar, key, "%.15g;"%init_params[key], verbose=False)

#Generate parfile and extract name of parfile
broydenparfile_output = subprocess.check_output(["perl", nsbh_id_rpar])
for line in broydenparfile_output.splitlines():
    if "Filename" in line:
        filename = line.split("=")[-1]
        filename = filename.replace(" ","")

#Move the parfile to corresponding path - Change this
#savepath = 'parfiles/InitialData'
#move('./%s'%filename, '%s/%s'%(savepath,filename))


In [ ]:
filename

## Run the broyden parfile to obtain the initial data 

In [ ]:
import time

start_time = time.time()
#Path where simulation results will be saved - 

#Change this
os.chdir('/nethome/bkhamesra3/Cactus_branches/NSBH_ID/')

parfile_path = '/nethome/bkhamesra3/Cactus_branches/Cactus_2019/par/NSBH/'

#Obtain parfile name and parfile path
parfile = os.path.join(parfile_path, filename)

#Run the code to get the ID output from TwoPunctures
broyden_output = subprocess.check_output(["../Cactus_2019/exe/cactus_Bowen_PS_Whisky", parfile])

#Remove all the checkpoints 
os.chdir('/nethome/bkhamesra3/Cactus_branches/NSBH_ID/')
check_direcs = glob.glob('./check*')
for direc in check_direcs:
    rmtree(direc)

#Save the output of simulation (stdout)
output_dir= filename.split('.par')[0]
stdout = open('./%s/stdout'%output_dir,'w')
stdout.write(broyden_output)
stdout.close()

print('Total Time taken = %g'%(time.time() - start_time))


## Generate the final par file for evolution

In [ ]:
#Find the last broyden iteration from output

last_iter_broyden=0
last_line_number=0
for i,line in enumerate(broyden_output.splitlines()):
    if "(Broyden): iteration #" in line:
        last_iter_broyden = int((line.split('#')[1])[0])
        last_line_number=i

if (last_iter_broyden==0):
    raise RuntimeError("Broyden Failed. Please run manually to check for errors.")
        
#Extract the final parameters from output
for i,line in enumerate(broyden_output.splitlines()):
    if i>last_line_number:
        
        if "(BowenID): TOV #1" in line:
            #These still include the scaling mass factors
            #bowen_tov_rho_cgs = float((line.split('rho_central = ')[1]).split('*')[0])
            bowen_tov_radius = float((line.split('Radius = ')[1]).split('*')[0])
        elif "(Broyden): Setting bowenid::tov_rho_central" in line:
            br_tov_rho_Kunits = float(line.split(' = ')[-1]) # Density in polytropic units
            br_tov_rho_M = br_tov_rho_Kunits/K_poly_M        # Density in Geometric units
            br_tov_rho_si = (br_tov_rho_M/Mscale_in_Msun**2)*((m_in_Msun**3)*Msun)
            print("Density (polytropic units) = %g, Density (SI) = %g"%(br_tov_rho_Kunits, br_tov_rho_si))#(br_tov_rho/Mscale_in_Msun**2)*((m_in_Msun**3)*Msun)))
        elif "(Broyden): Setting bowenid::bh_bare_mass" in line:
            br_punc_mass_M =  float(line.split(' = ')[-1])
            print("Puncture mass (in M) = %.8g"%(br_punc_mass_M))
            
            

# Warning - Do not read final masses of star and BH from stdout of Broyden. As we already provide target masses to 
# the most accurate value initially, our original values do not contain any error  and hence, would not induce
# any errors in other parameters when used for rescaling in rpar. 
        #elif "(Broyden): TOV Mass" in line:
        #    br_tov_mass = float(line.split(':')[-1])
        #elif "(Broyden): Found mass of black hole" in line:
        #    br_bh_mass = float(line.split('=')[-1])
                
print bowen_tov_radius*Mscale_in_Msun/m_in_Msun
bowen_tov_radius = bowen_tov_radius*Mscale_in_Msun/m_in_Msun
br_puncmass_Msun = br_punc_mass_M*Mscale_in_Msun


#Switch to rpar file paths
#Change this path
os.chdir('/nethome/bkhamesra3/Cactus_branches/Cactus_2019/par/NSBH')

#Define Evolution parfile paths
nsbh_evol_rpar = 'NSBH_Evolution_Python.rpar'
copyfile(nsbh_id_rpar, nsbh_evol_rpar)

#Replace the broyden parameters in rpar file
broyden_params = {"my $rhoStar_central_kgm3_br":br_tov_rho_si,
                  "my $puncmass_Msun_br":br_puncmass_Msun,
                  "my $radius_minus_m_br": bowen_tov_radius,
                  "my $params_from_broyden": 1}
for key in broyden_params.keys():
    print("%s: %.15g"%(key, broyden_params[key]))
    replace_data(nsbh_evol_rpar, key, "%.15g;"%broyden_params[key], verbose=False)


#Generate parfile and extract name of parfile
evolparfile_output = subprocess.check_output(["perl", nsbh_evol_rpar])


In [ ]:
    nn = 1./(gamma_poly-1)
kfactor = K_poly**nn
rho_dimless = 0.0782736
rhoc = rho_dimless/kfactor
print rhoc

In [ ]:
1./m_in_Msun

In [ ]:
 0.1666666666/0.9483460
    


In [ ]:

def construct_statictov(rho_init, K_poly):
    """Set up a Isotropic TOV star for a given central density
    """
    
    initdata = {}
    initdata['eqstate'] = eos_type

    gamma = 2
    K =  K_poly                                        # Msun**(2.)  

    rhoc = rho_init                                      # kg/m**3
    rhoc = rhoc/((m_in_Msun**3)*Msun)                    # Msun
   
    rho_atm = rhoc*1e-10
    press_atm = K*rho_atm**gamma

    n = 1./(gamma-1)
    delta_r = 1e-3                                       
    verbose=0

    initdata['init_mass'] = 0.
    initdata['init_restmass'] = 0.
    initdata['r_init'] = 0.
    initdata['theta'] = 1.0
    initdata['psi'] = 1.
    initdata['psider'] = 0.
    initdata['thetader'] = 0.

    initdata['K0'] = K
    initdata['n0'] = n
    initdata['K1'] = 0.
    initdata['n1'] = 0.

    initdata['rho_atm'] = rho_atm
    initdata['rho0'] = rhoc
    initdata['tov_type']='tov_bowen'
    initdata['delta_r'] = delta_r
    initdata['verbose'] = verbose

    if verbose>0:
        print("Polytropic Constants: K = %.3f (Msun^2), Gamma = %.3f"%(K, gamma))
        print("Stellar Density: rhoc = %g(/Msun^2), Pressure = %g(/Msun^2) \n"%(rhoc, K*rhoc**gamma))

    tov_bowen = bowenstar(initdata)

    press_iso, admmass_iso, restmass_iso, psi_iso, psider_iso, theta_iso, thetader_iso, riso_iso = tov_bowen.tovsolver()

    if press_iso[-1]<0:
        #print('Pressure at last index is negative \n')
    
        press_iso = press_iso[:-1]
        admmass_iso = admmass_iso[:-1]
        restmass_iso = restmass_iso[:-1]
        psi_iso = psi_iso[:-1]
        psider_iso = psider_iso[:-1]
        theta_iso = theta_iso[:-1]
        thetader_iso = thetader_iso[:-1]
        riso_iso = riso_iso[:-1]

    radius = riso_iso[-1]
    mass_iso = 2*radius*(psi_iso-1)
    return radius, mass_iso[-1]




In [ ]:
K_poly = 200

In [ ]:
logrho = np.arange(16,20, 0.1)
K_poly = 200.

radius, mass, density = [], [], []

j=0
for i in logrho:
    rad, m = construct_statictov(10**i, K_poly)
    radius.append(rad)
    mass.append(m)
    density.append(10**i)
    
    print("%d \t %g \t %g \t %g" %(j, rad, m, 10**i))
    j+=1



In [ ]:
import matplotlib.pyplot as plt

density_Msun = np.asarray(density)/((m_in_Msun**3)*Msun) 
mass_Msun = np.asarray(mass)
radius_Msun = np.asarray(radius)


In [ ]:
density_si = density_Msun*((m_in_Msun**3)*Msun) 
radius_si = radius_Msun/(m_in_Msun)

plt.figure(figsize=(15,8))
plt.subplot(121)
plt.semilogx(density_si, mass_Msun)
plt.xlabel(r'${\rho (kg/m^3)}$')
plt.ylabel(r'${M (M_\odot)}$')
plt.grid(True)
#plt.xlim(0,1)

plt.subplot(122)
plt.semilogx(density_si, radius_si/1000)
plt.xlabel(r'${\rho (kg/m^3)}$')
plt.ylabel(r'${R (km)}$')
plt.grid(True)
#plt.xlim(0,1)
plt.show()
plt.close()

plt.figure(figsize=(15,6))
plt.plot(radius_Msun, mass_Msun)
plt.xlabel(r'${R (M_{\odot})}$')
plt.ylabel(r'${M(M_{\odot})}$')
plt.grid(True)
plt.show()
plt.close()

plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(radius_Msun, mass_Msun/radius_Msun)
plt.xlabel(r'${R(M_{\odot})}$')
plt.ylabel(r'$Compactness$')
plt.grid(True)

plt.subplot(122)
plt.plot(mass_Msun, mass_Msun/radius_Msun)
plt.xlabel(r'${M(M_{\odot})}$')
plt.ylabel(r'$Compactness$')
plt.grid(True)
plt.show()
plt.close()

In [ ]:

plt.figure(figsize=(15,8))
plt.subplot(121)
plt.plot(density_Msun*K_poly, mass_Msun/K_poly**0.5)
plt.xlabel(r'$\bar{\rho}$')
plt.ylabel(r'$\bar{M}$')
plt.xlim(0,1)

plt.subplot(122)
plt.plot(density_Msun*K_poly, radius_Msun/K_poly**0.5)
plt.xlabel(r'$\bar{\rho}$')
plt.ylabel(r'$\bar{R}$')
plt.xlim(0,1)
plt.show()
plt.close()

plt.figure(figsize=(15,6))
plt.plot(radius_Msun/K_poly**0.5, mass_Msun/K_poly**0.5)
plt.axhline(y=0.1)
plt.xlabel(r'$\bar{R}$')
plt.ylabel(r'$\bar{M}$')
plt.show()
plt.close()


plt.figure(figsize=(15,6))
plt.subplot(121)
plt.axhline(y=0.15)
plt.plot(radius_Msun/K_poly**0.5, mass_Msun/radius_Msun)
plt.xlabel(r'$\bar{R}$')
plt.ylabel(r'$Compactness$')

plt.subplot(122)
plt.plot(mass_Msun/K_poly**0.5, mass_Msun/radius_Msun)
#plt.axvline(x=0.13)
plt.axhline(y=0.27)
#plt.xlim(0.09, 0.11)
plt.xlabel(r'$\bar{M}$')
plt.ylabel(r'$Compactness$')
plt.show()
plt.close()

In [ ]:
## BBH vs NSBH grid Structure

Msun = 1.989e30    #kg
kg_in_Msun = 1./Msun
m_in_Msun = c**2/G/Msun
s_in_Msun = c**3/G/Msun
#print m_in_Msun


Mstar = 2.5   #Msun
q = 10
Mbh = q*Mstar
Mtotal = Mstar+Mbh

bh1_mass = q/(1.+q)
bh2_mass = 1./(1.+q)


Star_res_m = 250
Star_res_Msun = 250*m_in_Msun
Star_res_M = Star_res_Msun/Mtotal



bbh_genres = Mtotal/140
sbh_res = 2*bbh_genres



print 1./Star_res_M, 1./sbh_res


In [ ]:
15*60./s_in_Msun

In [ ]:
L = 140.5
nlvl = 8
dx_f = 0.0305
dx_L = dx_f*2**(nlvl-1)
rf = L/2**(nlvl-1)
print("Rf = %g, dx_f = %g, Nf = %g"%(rf, dx_f, rf/dx_f))

In [ ]:
K = (1.35/0.1395)**2
M0 = 0.5534*K**0.5
mbh = 3*1.35
mbh_M0 = 3*1.35/M0
print 1./(dx_f/mbh_M0), 1./dx_f

rns = radius_ns_sch/M0
print rns/dx_f

In [ ]:
#os.chdir('/localdata2/bkhamesra3/simulations/Hive/NSBH/MassRatio_5/NSBH_MichaelSimulation/')
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
#plt.plot(t, re)

ylm = '/localdata2/bkhamesra3/simulations/Hive/NSBH/MassRatio_5/NSBH_MichaelSimulation/Ylm_WEYLSCAL4::Psi4_l2_m2_r70.00.asc'
t,re,im = np.loadtxt(ylm, unpack=True, usecols=(0,1,2))
amp = np.sqrt(re**2 + im**2)
tmaxamp = t[amp==np.amax(amp)]
plt.plot(t -tmaxamp, (re**2 + im**2)**0.5, label='M')


ylm = '/localdata2/bkhamesra3/simulations/Hive/NSBH/MassRatio_5/Trash/NSBH_q5_D8.8_q0_C0.17_M124/Summary/Data/Ylm_WEYLSCAL4::Psi4r_l2_m2_r70.00.asc'
t,re,im = np.loadtxt(ylm, unpack=True, usecols=(0,1,2))
amp = np.sqrt(re**2 + im**2)
tmaxamp = t[amp==np.amax(amp)]

plt.plot(t-tmaxamp, (re**2 + im**2)**0.5, label='BK')
plt.legend()
plt.xlim(-100,100)
plt.grid(True)
plt.show()

In [ ]:
 0.0138272*4.05/m_in_Msun